In [1]:
!pip install transformers==4.46.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 139.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 140.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2

In [2]:
import pandas as pd
import torch
import transformers
import sagemaker
from sagemaker.huggingface import HuggingFace
from sagemaker import get_execution_role

role = get_execution_role()

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
role

'arn:aws:iam::395709834669:role/service-role/AmazonSageMaker-ExecutionRole-20250420T152013'

In [3]:
import os
print(os.getcwd())

/home/sagemaker-user


In [6]:
sagemaker_session = sagemaker.Session()

In [12]:
import boto3

session = boto3.session.Session()
print("Region:", session.region_name)

s3 = boto3.client("s3")
try:
    s3.head_bucket(Bucket='hugging-face-multiclass-textclassification-bucket-custombucket')
    print("✅ Bucket exists and is accessible.")
except Exception as e:
    print("❌ Bucket issue:", e)


Region: eu-central-1
✅ Bucket exists and is accessible.


In [13]:
import boto3

sts = boto3.client("sts")
identity = sts.get_caller_identity()
print("IAM Role/User ARN:", identity["Arn"])


IAM Role/User ARN: arn:aws:sts::395709834669:assumed-role/AmazonSageMaker-ExecutionRole-20250420T152013/SageMaker


In [16]:
from sagemaker.huggingface import HuggingFace

print("Output path:", huggingface_estimator.output_path)
print("Entry point:", huggingface_estimator.entry_point)


Output path: s3://hugging-face-multiclass-textclassification-bucket-custombucket/output/
Entry point: script.py


In [15]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
                        entry_point = 'script.py',  # this will have the information about the model architecture, training data, loss  function, 
                        source_dir = './',
                        role = role,
                        instance_count = 1,
                        instance_type = 'ml.g4dn.xlarge',
                        transformers_version = '4.46.1',
                        pytorch_version = '2.3.0',
                        output_path = 's3://hugging-face-multiclass-textclassification-bucket-custombucket/output/',
                        py_version = 'py311',
                        hyperparameters = {'epochs':2,
                                          'train_batch_size':4,
                                          'val_batch_size':2,
                                          'learning_rate':1e-05
                                          },
                        enable_sagemaker_metrics = True)

In [18]:
huggingface_estimator.fit()

[04/20/25 19:06:38] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=623795;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=739343;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[04/20/25 19:07:43] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=542816;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=331325;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=595843;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=459042;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             huggingface-pytorch-training-2025-04-20-19-06-38-792                                  

2025-04-20 19:07:44 Starting - Starting the training job...
2025-04-20 19:07:58 Starting - Preparing the instances for training...
2025-04-20 19:08:40 Downloading - Downloading the training image...........................
2025-04-20 19:12:58 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-04-20 19:13:09,956 sagemaker-training-toolkit

In [20]:
!pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.5 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.242.0
    Uninstalling sagemaker-2.242.0:
      Successfully uninstalled sagemaker-2.242.0


In [22]:
transformers_version='4.28.1'

In [30]:
from sagemaker.huggingface import HuggingFaceModel

# S3 path where the trained model artifacts are stored
model_artifact = "s3://hugging-face-multiclass-textclassification-bucket-custombucket/output/huggingface-pytorch-training-2025-04-20-19-06-38-792/output/model.tar.gz"

# Create a HuggingFace model for deployment
huggingface_model = HuggingFaceModel(
    model_data=model_artifact,
    role=role,
    transformers_version='4.28.1',  # Use a supported version
    pytorch_version='2.0.0',  # Use a version of PyTorch supported by the SDK
    py_version='py310',  # Match the Python version used for training
)

# Deploy the model to a SageMaker endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'  # Choose an appropriate instance type
)


[04/20/25 19:35:45] INFO     Creating model with name:                                              ]8;id=18241;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=488885;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-04-20-19-35-45-435                                 

[04/20/25 19:35:46] INFO     Creating endpoint-config with name                                     ]8;id=910994;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=590239;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\
                             huggingface-pytorch-inference-2025-04-20-19-35-46-087                                 

                    INFO     Creating endpoint with name                                            ]8;id=757075;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=610837;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\
                             huggingface-pytorch-inference-2025-04-20-19-35-46-087                                 

------------!

In [32]:
response = predictor.predict({"inputs": "Tesla Reports Record Quarterly Earnings Amid Surge in EV Demand"})
print(response)

[{'label': 'LABEL_2', 'score': 0.598465621471405}]


In [35]:
import boto3
from sagemaker import get_execution_role
from sagemaker import Session

# Initialize boto3 client and sagemaker session
sagemaker_client = boto3.client('sagemaker')
sagemaker_session = Session()

# Get the current training job name (from an existing estimator)
job_name = huggingface_estimator.latest_training_job.name

# Stop the current training job if it is running
# sagemaker_client.stop_training_job(TrainingJobName=job_name)
print(f"Stopped training job: {job_name}")

# 1. Delete all SageMaker endpoints
def delete_all_endpoints():
    # List all endpoints
    endpoints = sagemaker_client.list_endpoints(StatusEquals='InService')['Endpoints']
    if not endpoints:
        print("No endpoints found.")
        return
    
    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        print(f"Deleting endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Deleted endpoint: {endpoint_name}")

# 2. Terminate all running training jobs (terminate the instances too)
def terminate_all_training_jobs():
    # List all training jobs (in any state)
    training_jobs = sagemaker_client.list_training_jobs(StatusEquals='InProgress')['TrainingJobSummaries']
    
    if not training_jobs:
        print("No training jobs are running.")
        return
    
    for job in training_jobs:
        job_name = job['TrainingJobName']
        print(f"Stopping training job: {job_name}")
        sagemaker_client.stop_training_job(TrainingJobName=job_name)
        print(f"Stopped training job: {job_name}")

# 3. Delete all SageMaker models (cleanup)
def delete_all_models():
    # List all models
    models = sagemaker_client.list_models()['Models']
    if not models:
        print("No models found.")
        return
    
    for model in models:
        model_name = model['ModelName']
        print(f"Deleting model: {model_name}")
        sagemaker_client.delete_model(ModelName=model_name)
        print(f"Deleted model: {model_name}")

# 4. Terminate any running inference instances (endpoint instances)
def terminate_inference_instances():
    # List all running endpoints
    endpoints = sagemaker_client.list_endpoints(StatusEquals='InService')['Endpoints']
    
    if not endpoints:
        print("No inference endpoints are running.")
        return
    
    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        print(f"Deleting inference endpoint: {endpoint_name}")
        sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Deleted inference endpoint: {endpoint_name}")

# Execute the cleanup functions
delete_all_endpoints()        # Delete all endpoints
# terminate_all_training_jobs()  # Terminate all training jobs
delete_all_models()            # Delete all models
terminate_inference_instances() # Terminate inference instances

print("Cleanup completed.")


Stopped training job: huggingface-pytorch-training-2025-04-20-19-06-38-792
Deleting endpoint: huggingface-pytorch-inference-2025-04-20-19-35-46-087
Deleted endpoint: huggingface-pytorch-inference-2025-04-20-19-35-46-087
Deleting model: huggingface-pytorch-inference-2025-04-20-19-35-45-435
Deleted model: huggingface-pytorch-inference-2025-04-20-19-35-45-435
Deleting model: huggingface-pytorch-inference-2025-04-20-19-35-12-964
Deleted model: huggingface-pytorch-inference-2025-04-20-19-35-12-964
Deleting model: xgboost-2025-04-20-16-06-29-820
Deleted model: xgboost-2025-04-20-16-06-29-820
No inference endpoints are running.
Cleanup completed.
